<a href="https://colab.research.google.com/github/Navjotkhatri/Productionization_of_ML_Systems_in_Travel_Industry/blob/main/REST_API_for_Flight_Price_Regression_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2. REST API for Regression Model:**

**Develop a REST API using Flask to serve the flight price prediction model, enabling real-time price predictions.**

In [51]:
!pip install Flask pyngrok


In [61]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import os
import pickle

In [62]:
import os
import glob
import pickle
import numpy as np
import pandas as pd
from pyngrok import ngrok
from datetime import datetime
from google.colab import drive
from flask_ngrok import run_with_ngrok
from sklearn.preprocessing import StandardScaler
from flask import Flask, render_template, request, jsonify

In [53]:
app = Flask(__name__)


In [54]:
@app.route('/', methods=['GET'])
def home():
    return "Hello, this is your Flask app running on ngrok!"


In [56]:
# Set your Ngrok auth token
NGROK_AUTH_TOKEN = "2kEiDvlDvDmlSrjFdidw1UnLQmx_32guShAgCSJ9W3gqsYXPA"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open a tunnel on the default port 5000
public_url = ngrok.connect(5000)
print(f"Your Flask app is running on {public_url}")

# Start the Flask app
app.run()


Your Flask app is running on NgrokTunnel: "https://7131-35-243-249-27.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [08/Aug/2024 10:14:16] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Aug/2024 10:14:16] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
!ngrok authtoken "2kEiDvlDvDmlSrjFdidw1UnLQmx_32guShAgCSJ9W3gqsYXPA"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [63]:
def predict_price(input_data,model):
    # Prepare the input data
    #text_columns = ['from','to','']

    # Initialize an empty DataFrame
    df = pd.DataFrame([input_data])

    # Encode text-based columns and create embeddings
    for column in input_data:
        df[column + '_embedding'] = df[column].apply(lambda text: model.encode(text))



    # Make predictions using the trained Linear Regression model
    y_pred = model.predict(df)

    return y_pred[0]

In [87]:
!pip install scikit-learn

import pandas as pd
from sklearn.preprocessing import LabelEncoder

def predict_price(input_data,model):
    # Prepare the input data
    text_columns = ['from','to','flightType','agency']
    numerical_columns = ['month', 'speed', 'weekday_num', 'year']  # Identify numerical columns

    # Initialize an empty DataFrame
    df = pd.DataFrame([input_data])

    # Access the best estimator from GridSearchCV
    best_model = model.best_estimator_

    # Encode text-based columns using LabelEncoder
    for column in text_columns:
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])

    # Convert numerical columns to appropriate data type (e.g., int)
    for column in numerical_columns:
        df[column] = df[column].astype(int)  # Convert to integer

    y_q1 = best_model.predict(df) # Use best_model here

    return y_q1[0]


In [74]:
input_date={"agency":"FlyingDrops","flightType":"firstClass","from":"Aracaju (SE)","month":"10","speed":"300","to":"Brasilia (DF)","weekday_num":"0","year":"2023"}

In [75]:
# Open the file in read-binary ('rb') mode
with open('/content/flight_price_prediction_regression.pkl', 'rb') as file:
    model = pickle.load(file)

In [88]:
predict_price(input_date,model)

1401.4756

In [90]:
NGROK_AUTH_TOKEN = "2kEiDvlDvDmlSrjFdidw1UnLQmx_32guShAgCSJ9W3gqsYXPA"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

app = Flask(__name__)
#run_with_ngrok(app)

# Open the file in read-binary ('rb') mode
with open('/content/flight_price_prediction_regression.pkl', 'rb') as file:
    model = pickle.load(file)

@app.route('/', methods=['GET', 'POST'])
def predict():
    return """

<!DOCTYPE html>
<html>

<head>
    <title>Flight Price Prediction</title>
    <style>
        body {
            font-family: 'Poppins', sans-serif;
            background-color: #f9f9f9;
            margin: 0;
            padding: 0;
        }

        .container {
            max-width: 800px;
            margin: 0 auto;
            padding: 40px;
            background-color: #ffffff;
            border-radius: 10px;
            box-shadow: 0 10px 20px rgba(0, 0, 0, 0.1);
            text-align: center;
        }

        h1 {
            color: #184344;
            font-size: 36px;
            margin-bottom: 20px;
        }

        form {
            text-align: left;
        }

        input[type="text"],
        input[type="number"] {
            width: 100%;
            padding: 15px;
            margin: 15px 0;
            border: none;
            border-bottom: 2px solid #184344;
            font-size: 18px;
            background-color: transparent;
            color: #333;
            transition: border-bottom 0.3s ease;
        }

        input[type="text"]:focus,
        input[type="number"]:focus {
            border-bottom: 2px solid #184344;
            outline: none;
        }

        input[type="checkbox"] {
            margin-right: 10px;
        }

        input[type="submit"] {
            background-color: #184344;
            color: #fff;
            padding: 15px 30px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 20px;
            transition: background-color 0.3s ease;
        }

        input[type="submit"]:hover {
            background-color: #184344;
        }

        p#prediction {
            margin-top: 20px;
            font-size: 24px;
            color: #184344;
        }
    </style>
</head>

<body>
    <div class="container">
        <h1>Flight Price Prediction</h1>
        <form action="/predict" method="POST">

            <label>Departure City:</label><br><br>

            <input type="checkbox" name="From" value="Aracaju (SE)">Aracaju (SE)<br>
            <input type="checkbox" name="From" value="Brasilia (DF)">Brasilia (DF)<br>
            <input type="checkbox" name="From" value="Campo Grande (MS)">Campo Grande (MS)<br>
            <input type="checkbox" name="From" value="Florianopolis (SC)">Florianopolis (SC)<br>
            <input type="checkbox" name="From" value="Natal (RN)">Natal (RN)<br>
            <input type="checkbox" name="From" value="Recife (PE)">Recife (PE)<br>
            <input type="checkbox" name="From" value="Rio de Janeiro (RJ)">Rio de Janeiro (RJ)<br>
            <input type="checkbox" name="From" value="Salvador (BH)">Salvador (BH)<br>
            <input type="checkbox" name="From" value="Sao Paulo (SP)">Sao Paulo (SP)<br><br>



            <label>Destination City:</label><br><br>

            <input type="checkbox" name="To" value="Aracaju (SE)">Aracaju (SE)<br>
            <input type="checkbox" name="To" value="Brasilia (DF)">Brasilia (DF)<br>
            <input type="checkbox" name="To" value="Campo Grande (MS)">Campo Grande (MS)<br>
            <input type="checkbox" name="To" value="Florianopolis (SC)">Florianopolis (SC)<br>
            <input type="checkbox" name="To" value="Natal (RN)">Natal (RN)<br>
            <input type="checkbox" name="To" value="Recife (PE)">Recife (PE)<br>
            <input type="checkbox" name="To" value="Rio de Janeiro (RJ)">Rio de Janeiro (RJ)<br>
            <input type="checkbox" name="To" value="Salvador (BH)">Salvador (BH)<br>
            <input type="checkbox" name="To" value="Sao Paulo (SP)">Sao Paulo (SP)<br><br>

            <label>Flight Type:</label><br><br>

            <input type="checkbox" name="flightType" value="economic"> Economic<br>
            <input type="checkbox" name="flightType" value="firstClass"> FirstClass<br>
            <input type="checkbox" name="flightType" value="premium"> Premium<br><br>


            <label>Agency:</label><br><br>


            <input type="checkbox" name="agency" value="CloudFy"> CloudFy<br>
            <input type="checkbox" name="agency" value="FlyingDrops"> FlyingDrops<br>
            <input type="checkbox" name="agency" value="Rainbow"> Rainbow<br><br>


            <label for="weekday_num">Weekday (0=Sunday, 6=Saturday):</label>
            <input type="number" name="weekday_num" min="0" max="6" placeholder="Day of the week"><br>

            <label for="month">Month:</label>
            <input type="number" name="month" min="1" max="12" placeholder="Month"><br>

            <label for="year">Year:</label>
            <input type="number" name="year" min="2023" max="2123" placeholder="Year"><br>

            <label for="speed">Speed (km/h):</label>
            <input type="number" name="speed" placeholder="Speed"><br><br>

            <input type="submit" value="Predict">
        </form>
        <p id="prediction"></p>
    </div>
</body>

</html>


    """

@app.route('/predict', methods=['POST'])
def index():
    if request.method == 'POST':
        # Get input data from the form
        From = request.form.get('From')
        To = request.form.get('To')
        flighttype = request.form.get('flightType')
        agency = request.form.get('agency')
        weekday_num = request.form.get('weekday_num')
        month = request.form.get('month')
        year = request.form.get('year')
        speed = request.form.get('speed')

        # Create a dictionary to store the input data
        data = {
            'from': From,
            'to': To,
            'flightType': flighttype,
            'agency': agency,
            'weekday_num': weekday_num,
            'month': month,
            'year': year,
            'speed': speed
        }

        # Perform prediction using the custom_input dictionary
        prediction = predict_price(data,model)//10
        prediction = str(prediction)[-7:]

        return jsonify({'prediction': prediction})

# Open a tunnel on the default port 5000
public_url = ngrok.connect(5000)
print(f"Your Flask app is running on {public_url}")

if __name__ == "__main__":
    app.run()

Your Flask app is running on NgrokTunnel: "https://5b92-35-243-249-27.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [08/Aug/2024 10:55:30] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Aug/2024 10:55:31] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Aug/2024 10:55:52] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Aug/2024 10:56:14] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Aug/2024 10:56:38] "POST /predict HTTP/1.1" 200 -
